In [ ]:
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"

# os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/LLM'
# os.environ['HF_DATASETS_CACHE'] = '/content/drive/MyDrive/LLM'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

### Transformers

In [ ]:
# Setup libs
!pip -q install bitsandbytes accelerate xformers einops langchain faiss-cpu transformers sentence-transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig,pipeline,BitsAndBytesConfig

model_path = "mistralai/Mistral-7B-v0.3" # c'est juste une exemple 
token="Votre_token_Huggingface"


# Setup le tokeniseur
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True, token=token)


# Parametrer la config
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, token=token)
config.init_device = "cuda"
config.temperature = 0.1
# config.max_length =300
# config.eos_token_id=tokenizer.eos_token_id
# config.pad_token_id=tokenizer.pad_token_id
# config.do_sample = True

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )


model = AutoModelForCausalLM.from_pretrained(
    model_path,quantization_config=bnb_config,
    config=config,
    trust_remote_code=True , token=token
)

model.eval()

# pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)#, device=0,)

### Langchain

In [ ]:
!pip -q install langchain sentence-transformers openai tiktoken faiss-cpu

In [ ]:
import transformers
from langchain.llms import HuggingFacePipeline

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
my_pipeline = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
PROMPT = "### Question:\n{instruction}\n\n### Répondre:"

input_prompt = PROMPT.format_map(
    {"instruction": "Combien de types de fruits existe-t-il ?"}
)
my_pipeline(input_prompt)

In [ ]:
# Creer Prompt template
from langchain import PromptTemplate

template = "### Question:\n{question}\n\n### Répondre:"
prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt,
                     llm=my_pipeline
                     )

question = "Combien de côtés a un triangle ?"

result = llm_chain.run({"question":question})
print(result)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
from langchain.vectorstores import FAISS

In [ ]:
!pip -q install InstructorEmbedding gpt4all

In [ ]:
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings, GPT4AllEmbeddings
from langchain.vectorstores import FAISS
embeddings = GPT4AllEmbeddings()#OpenAIEmbeddings()#HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
#HuggingFaceEmbeddings(model_name=model_path)

In [ ]:
raw_text ="""
Votre Text 1
"""
raw_text ="""
Votre Text 2
"""
# obtenir les morceaux de texte
text_chunks = get_text_chunks(raw_text)

# creer vecteur store
#vectorstore = get_vectorstore(text_chunks)
vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)

In [ ]:
db = vectorstore.as_retriever(search_kwargs={'k': 3})

In [ ]:
from langchain import PromptTemplate

template_qa = "Utilisez le contexte suivant pour répondre à la question\n{context}\n### Question:\n{question}\n\n### Répondre:"
prompt_qa = PromptTemplate(template=template_qa, input_variables=["question"])

In [ ]:
from langchain import PromptTemplate
template = prompt = """<|im_start|>system
Utilisez le contexte suivant pour répondre à la question\n{context}\n
<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant"""

prompt_qa = PromptTemplate(template=template, input_variables=["question"])


In [ ]:
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

qa_chain = RetrievalQA.from_llm(llm=mistral_llm,
                                      retriever=db,
                                       return_source_documents=True,verbose=True,
                                prompt = prompt_qa
                                        )


In [ ]:
query = "La question est en rapport avec votre texte"

sol=qa_chain({"query": query})
print(sol)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt)


### Embedding

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain import PromptTemplate

template_qah = "Utilisez le contexte suivant pour répondre à la question\n{context}\net historique\n{chat_history}\n### Question:\n{question}\n\n### Répondre:"
prompt_qah = PromptTemplate(template=template_qah, input_variables=["question"])

template_qah_1 = "Historique:\n{chat_history}\n### Question:\n{question}\n\n### Répondre:"
prompt_qah_1 = PromptTemplate(template=template_qah_1, input_variables=["question"])

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain

qah_chain = ConversationalRetrievalChain.from_llm(llm=hf_pipeline,
                                      retriever=db,
                                       return_source_documents=False,verbose=True,
                                 memory = memory,combine_docs_chain_kwargs={'prompt': prompt_qah},
     condense_question_prompt=prompt_qah_1,
                                        )


In [ ]:
query = "La question est en rapport avec votre texte"

sol=qah_chain({"question": query})
print(sol)